In [1]:
!pip install bpemb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
import json
with open('/content/drive/MyDrive/ar.train.json', 'r') as file:
    df = json.load(file)

In [5]:
len(df)


45200

In [45]:
meanings = []
words = []
emb_sgns = []
emb_w2v = []
for js in df:
  meanings.append(js["gloss"])
  words.append(js["word"])
  emb_sgns.append(js["sgns"])
  emb_w2v.append(js["sgns"])

In [6]:
# class FFN(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(FFN, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         return x
#         device = "cuda"

In [43]:
import numpy as np
from bpemb import BPEmb
dim = 300
V_S=10000
bpemb = BPEmb(lang="ar", dim=dim, vs=V_S)

downloading https://nlp.h-its.org/bpemb/ar/ar.wiki.bpe.vs10000.d300.w2v.bin.tar.gz


100%|██████████| 11198290/11198290 [00:01<00:00, 11056737.64B/s]


In [9]:
# def return_emb(defn):
#   tokens =bpemb.encode(defn)
#   # list of tokens rep the defenation
#   emb = torch.zeros(100)
#   for token in tokens:
#     emb_tensor = bpemb.emb[token]
#     emb += emb_tensor

#   return emb/len(tokens)

In [27]:
import numpy as np

In [46]:
def return_emb(defn):
    tokens =bpemb.encode(defn)
    emb = np.zeros(300)
    count = 0
    for token in tokens:
        if token:
            try:
                emb_tensor = bpemb.emb[token]
                emb = emb + emb_tensor
                count += 1
            except KeyError:
                # handle case where token is not in vocabulary
                pass
    if count > 0:
        emb = emb / count
    return emb

In [47]:
emb_list = []

for i in range(len(meanings)):
    emb_m = return_emb(meanings[i])
    emb_list.append(emb_m)


sum_emb = sum(emb_list)

In [48]:
emb_sgns = np.array(emb_sgns)

In [49]:
emb_list = np.array(emb_list)

In [51]:
from keras.models import Sequential
from keras.layers import Dense

# Define the Keras model
model = Sequential()
model.add(Dense(1024, input_dim=300, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(300, activation=None))  # assuming output size is 100

# Compile the model
model.compile(loss='mean_squared_error', optimizer='sgd')

model.fit(x=emb_list, y=emb_sgns, epochs=10, batch_size=1)

# Evaluate the model
loss = model.evaluate(x=emb_list, y=emb_sgns)
print(f"the average loss is {loss}")

Epoch 1/10
45200/45200 [==============================] - 238s 5ms/step - loss: 0.0347
Epoch 2/10
45200/45200 [==============================] - 212s 5ms/step - loss: 0.0335
Epoch 3/10
45200/45200 [==============================] - 216s 5ms/step - loss: 0.0334
Epoch 4/10
45200/45200 [==============================] - 208s 5ms/step - loss: 0.0333
Epoch 5/10
45200/45200 [==============================] - 204s 5ms/step - loss: 0.0332
Epoch 6/10
45200/45200 [==============================] - 206s 5ms/step - loss: 0.0331
Epoch 7/10
45200/45200 [==============================] - 192s 4ms/step - loss: 0.0330
Epoch 8/10
45200/45200 [==============================] - 206s 5ms/step - loss: 0.0329
Epoch 9/10
45200/45200 [==============================] - 196s 4ms/step - loss: 0.0328
Epoch 10/10
1413/1413 [==============================] - 5s 4ms/step - loss: 0.0327
the average loss is 0.03269178420305252


In [4]:
import pandas as pd
df = pd.json_normalize(df)

In [5]:
df

,id,word,pos,gloss,electra,enId,sgns
0,ar.297,أخير,AA,عكس أوَّل، ليس بعده شيء,"[-0.6090424061, -1.3187981844, 0.1403246224, 0...",[],"[0.0231062472, 0.2055995017, 0.3978048265, -0...."
1,ar.65800,نَمِيمَة,noun,وشاية، نقل الحديث على وجه الإفساد والوقيعة بين...,"[0.4870503545, -1.1256181002, 0.8570457697, -0...",[],"[-0.0643204302, 0.1027905568, 0.3520267606, 0...."
2,ar.65790,نمَّ بين الزُّملاء,V,حرَّش وأغرى,"[-0.1836982568, -0.5688381866, 0.3699473242, 0...",[],"[-0.2224979252, 0.3184450567, 0.0620564818, 0...."
3,ar.38472,مفتاح,noun,ما يساعد على فهم مسألة معقدة، أو إيجاد مخرج لص...,"[-0.8474017382, -1.2676231861, 0.0681600049, -...",[],"[-0.3391492069, 0.0098403152, 0.0773706064, -0..."
4,ar.6547,أجحف بخصمه,V,اشتدّ في الإضرار به,"[0.3306163251, 0.2324001733, 0.3323483765, 0.1...",[],"[0.0974059105, 0.133708939, 0.322111696, 0.347..."
...,...,...,...,...,...,...,...
45195,ar.28481,مُشْكِلة,noun,صعوبة يجب تذليلها للحصول على نتيجة ما.,"[-1.4643167257, 0.0131763713, 0.5112231374, -0...","[en.7110, en.50868]","[0.299158603, 0.0243165549, 0.3381068707, -0.2..."
45196,ar.45986,لُبَد,noun,كثير متراكم,"[0.2182594091, -0.9905560017, -0.6166527271, -...",[en.6047],"[-0.0629788116, 0.1784198582, 0.2217850536, 0...."
45197,ar.25830,أهل السُّنَّة,noun,السَّلف الصَّالح، العاملون بالكتاب والسُّنَّة...,"[-0.6276198924, -0.239099741, 0.1758998334, -0...",[en.5907],"[-0.3708837032, 0.0437819362, 0.6107559204, 0...."
45198,ar.45784,كيلو سُعر,noun,وحدة قياس كمِّيَّة الحرارة التي تلزم لرفع درجة...,"[-1.5296031237, -0.1864113323, -0.0574643016, ...",,"[-0.0871081129, 0.066735819, 0.2245120704, 0.2..."
